In [53]:
import pandas as pd
import numpy as np

In [54]:
file_path = "../data/employee_raw_data.csv"
df = pd.read_csv(file_path)

In [55]:
# Display the first 10 rows of the data before cleaning
pd.set_option('display.max_columns', None)
df.head(10)

,EmpID,FirstName,LastName,StartDate,ExitDate,Title,Supervisor,ADEmail,BusinessUnit,EmployeeStatus,EmployeeType,PayZone,EmployeeClassificationType,TerminationType,TerminationDescription,DepartmentType,Division,DOB,State,JobFunctionDescription,GenderCode,LocationCode,RaceDesc,MaritalDesc,Performance Score,Current Employee Rating
0,3427,Uriah,Bridges,20-Sep-19,NaN,Production Technician I,Peter Oneill,uriah.bridges@bilearner.com,CCDR,Active,Contract,Zone C,Temporary,Unk,NaN,Production,Finance & Accounting,07-10-1969,MA,Accounting,Female,34904,White,Widowed,Fully Meets,4
1,3428,Paula,Small,11-Feb-23,NaN,Production Technician I,Renee Mccormick,paula.small@bilearner.com,EW,Active,Contract,Zone A,Part-Time,Unk,NaN,Production,Aerial,30-08-1965,MA,Labor,Male,6593,Hispanic,Widowed,Fully Meets,3
2,3429,Edward,Buck,10-Dec-18,NaN,Area Sales Manager,Crystal Walker,edward.buck@bilearner.com,PL,Active,Full-Time,Zone B,Part-Time,Unk,NaN,Sales,General - Sga,06-10-1991,MA,Assistant,Male,2330,Hispanic,Widowed,Fully Meets,4
3,3430,Michael,Riordan,21-Jun-21,NaN,Area Sales Manager,Rebekah Wright,michael.riordan@bilearner.com,CCDR,Active,Contract,Zone A,Full-Time,Unk,NaN,Sales,Finance & Accounting,04-04-1998,ND,Clerk,Male,58782,Other,Single,Fully Meets,2
4,3431,Jasmine,Onque,29-Jun-19,NaN,Area Sales Manager,Jason Kim,jasmine.onque@bilearner.com,TNS,Active,Contract,Zone A,Temporary,Unk,NaN,Sales,General - Con,29-08-1969,FL,Laborer,Female,33174,Other,Married,Fully Meets,3
5,3432,Maruk,Fraval,17-Jan-20,NaN,Area Sales Manager,Sheri Campos,maruk.fraval@bilearner.com,BPC,Active,Contract,Zone B,Full-Time,Unk,NaN,Sales,Field Operations,03-04-1949,CT,Driver,Male,6050,Black,Married,Fully Meets,3
6,3433,Latia,Costa,06-Apr-22,03-Jul-23,Area Sales Manager,Jacob Braun,latia.costa@bilearner.com,WBL,Active,Full-Time,Zone B,Temporary,Involuntary,Me see picture nature degree benefit.,Sales,General - Eng,01-07-1942,CA,Technician,Female,90007,Hispanic,Divorced,Exceeds,4
7,3434,Sharlene,Terry,06-Nov-20,29-Jan-23,Area Sales Manager,Tracy Marquez,sharlene.terry@bilearner.com,CCDR,Active,Contract,Zone C,Full-Time,Involuntary,Blue community type skill story.,Sales,Engineers,07-03-1957,OR,Engineer,Female,97756,White,Divorced,Fully Meets,2
8,3435,Jac,McKinzie,18-Aug-18,NaN,Area Sales Manager,Sharon Becker,jac.mckinzie@bilearner.com,NEL,Active,Contract,Zone B,Part-Time,Unk,NaN,Sales,Executive,15-05-1974,TX,Executive Assistant,Male,78789,Black,Widowed,Exceeds,3
9,3436,Joseph,Martins,21-Jan-22,29-Jun-23,Area Sales Manager,George Jenkins,joseph.martins@bilearner.com,BPC,Active,Part-Time,Zone B,Temporary,Resignation,Summer personal bag.,Sales,Engineers,11-11-1949,TX,Engineer,Male,78207,Asian,Widowed,Fully Meets,5


In [56]:
# Check for duplicates and drop if any
df.drop_duplicates(inplace=True)

In [57]:
# Check missing values
missing_values = df.isnull().sum()
print(f"Missing values per column:\n{missing_values}")

Missing values per column:
EmpID                            0
FirstName                        0
LastName                         0
StartDate                        0
ExitDate                      1467
Title                            0
Supervisor                       0
ADEmail                          0
BusinessUnit                     0
EmployeeStatus                   0
EmployeeType                     0
PayZone                          0
EmployeeClassificationType       0
TerminationType                  0
TerminationDescription        1467
DepartmentType                   0
Division                         0
DOB                              0
State                            0
JobFunctionDescription           0
GenderCode                       0
LocationCode                     0
RaceDesc                         0
MaritalDesc                      0
Performance Score                0
Current Employee Rating          0
dtype: int64


In [58]:
# Fill missing ExitDate with 'Active' and drop TerminationDescription
df['ExitDate'] = df['ExitDate'].fillna('Active')
df.drop(columns=['TerminationDescription'], inplace=True)

# Replace 'Unk' with 'Unknown' in TerminationType
df['TerminationType'] = df['TerminationType'].replace('Unk', 'Unknown')

# Update TerminationType to 'Still Employed' for active employees with 'Unknown' status
df.loc[(df['TerminationType'] == 'Unknown') & (df['EmployeeStatus'] == 'Active'), 'TerminationType'] = 'Still Employed'

# Clean DOB and convert to datetime
df['DOB'] = pd.to_datetime(df['DOB'].str.strip(), errors='coerce', dayfirst=True)

# Calculate Age
today = pd.to_datetime('today')
df['Age'] = (today - df['DOB']).dt.days // 365

# Convert StartDate and ExitDate to datetime, format ExitDate if not 'Active'
df['StartDate'] = pd.to_datetime(df['StartDate'], format='%d-%b-%y', errors='coerce')
df['ExitDate'] = pd.to_datetime(df['ExitDate'], format='%d-%b-%y', errors='coerce').dt.date
df['ExitDate'] = df['ExitDate'].where(df['ExitDate'].notna(), 'Active')

# Calculate Employment Duration
def calculate_employment_duration(row):
    end_date = pd.to_datetime(row['ExitDate']) if row['ExitDate'] != 'Active' else today
    start_date = pd.to_datetime(row['StartDate'])
    if pd.isna(start_date) or pd.isna(end_date): return None
    delta = end_date - start_date
    years = delta.days // 365
    months = (delta.days % 365) // 30
    return f'{years} years {months} months'

df['EmploymentDuration'] = df.apply(calculate_employment_duration, axis=1)

In [59]:
# Check unique values in EmploymentDuration to ensure validity
print(df['EmploymentDuration'].unique())

['5 years 6 months' '2 years 1 months' '6 years 3 months'
 '3 years 9 months' '5 years 9 months' '5 years 2 months'
 '1 years 2 months' '2 years 2 months' '6 years 7 months'
 '1 years 5 months' '1 years 7 months' '0 years 6 months'
 '3 years 2 months' '5 years 11 months' '2 years 11 months'
 '0 years 2 months' '4 years 2 months' '3 years 7 months'
 '3 years 0 months' '1 years 1 months' '0 years 0 months'
 '2 years 3 months' '1 years 8 months' '2 years 4 months'
 '2 years 5 months' '2 years 6 months' '1 years 0 months'
 '2 years 0 months' '4 years 6 months' '5 years 1 months'
 '3 years 4 months' '0 years 12 months' '4 years 10 months'
 '0 years 11 months' '5 years 0 months' '1 years 9 months'
 '1 years 6 months' '0 years 3 months' '0 years 8 months'
 '0 years 1 months' '1 years 3 months' '3 years 6 months'
 '3 years 5 months' '0 years 7 months' '6 years 1 months'
 '5 years 4 months' '5 years 8 months' '4 years 9 months'
 '3 years 3 months' '5 years 5 months' '4 years 7 months'
 '3 years

In [60]:
# Check unique values in Age to ensure validity
print(df['Age'].unique())

[55 59 33 26 76 82 68 50 75 61 77 43 73 35 72 30 41 39 28 66 32 57 78 42
 81 27 63 46 37 80 58 74 67 79 62 60 29 34 31 71 51 40 44 54 25 65 48 36
 52 47 83 70 24 64 56 38 53 45 49 23 69]


In [61]:
# Check the first 10 rows to review the final result after cleaning
df.head(10)

,EmpID,FirstName,LastName,StartDate,ExitDate,Title,Supervisor,ADEmail,BusinessUnit,EmployeeStatus,EmployeeType,PayZone,EmployeeClassificationType,TerminationType,DepartmentType,Division,DOB,State,JobFunctionDescription,GenderCode,LocationCode,RaceDesc,MaritalDesc,Performance Score,Current Employee Rating,Age,EmploymentDuration
0,3427,Uriah,Bridges,2019-09-20,Active,Production Technician I,Peter Oneill,uriah.bridges@bilearner.com,CCDR,Active,Contract,Zone C,Temporary,Still Employed,Production,Finance & Accounting,1969-10-07,MA,Accounting,Female,34904,White,Widowed,Fully Meets,4,55,5 years 6 months
1,3428,Paula,Small,2023-02-11,Active,Production Technician I,Renee Mccormick,paula.small@bilearner.com,EW,Active,Contract,Zone A,Part-Time,Still Employed,Production,Aerial,1965-08-30,MA,Labor,Male,6593,Hispanic,Widowed,Fully Meets,3,59,2 years 1 months
2,3429,Edward,Buck,2018-12-10,Active,Area Sales Manager,Crystal Walker,edward.buck@bilearner.com,PL,Active,Full-Time,Zone B,Part-Time,Still Employed,Sales,General - Sga,1991-10-06,MA,Assistant,Male,2330,Hispanic,Widowed,Fully Meets,4,33,6 years 3 months
3,3430,Michael,Riordan,2021-06-21,Active,Area Sales Manager,Rebekah Wright,michael.riordan@bilearner.com,CCDR,Active,Contract,Zone A,Full-Time,Still Employed,Sales,Finance & Accounting,1998-04-04,ND,Clerk,Male,58782,Other,Single,Fully Meets,2,26,3 years 9 months
4,3431,Jasmine,Onque,2019-06-29,Active,Area Sales Manager,Jason Kim,jasmine.onque@bilearner.com,TNS,Active,Contract,Zone A,Temporary,Still Employed,Sales,General - Con,1969-08-29,FL,Laborer,Female,33174,Other,Married,Fully Meets,3,55,5 years 9 months
5,3432,Maruk,Fraval,2020-01-17,Active,Area Sales Manager,Sheri Campos,maruk.fraval@bilearner.com,BPC,Active,Contract,Zone B,Full-Time,Still Employed,Sales,Field Operations,1949-04-03,CT,Driver,Male,6050,Black,Married,Fully Meets,3,76,5 years 2 months
6,3433,Latia,Costa,2022-04-06,2023-07-03,Area Sales Manager,Jacob Braun,latia.costa@bilearner.com,WBL,Active,Full-Time,Zone B,Temporary,Involuntary,Sales,General - Eng,1942-07-01,CA,Technician,Female,90007,Hispanic,Divorced,Exceeds,4,82,1 years 2 months
7,3434,Sharlene,Terry,2020-11-06,2023-01-29,Area Sales Manager,Tracy Marquez,sharlene.terry@bilearner.com,CCDR,Active,Contract,Zone C,Full-Time,Involuntary,Sales,Engineers,1957-03-07,OR,Engineer,Female,97756,White,Divorced,Fully Meets,2,68,2 years 2 months
8,3435,Jac,McKinzie,2018-08-18,Active,Area Sales Manager,Sharon Becker,jac.mckinzie@bilearner.com,NEL,Active,Contract,Zone B,Part-Time,Still Employed,Sales,Executive,1974-05-15,TX,Executive Assistant,Male,78789,Black,Widowed,Exceeds,3,50,6 years 7 months
9,3436,Joseph,Martins,2022-01-21,2023-06-29,Area Sales Manager,George Jenkins,joseph.martins@bilearner.com,BPC,Active,Part-Time,Zone B,Temporary,Resignation,Sales,Engineers,1949-11-11,TX,Engineer,Male,78207,Asian,Widowed,Fully Meets,5,75,1 years 5 months


In [62]:
# Summary Statistics and Data Check
df.describe()

,EmpID,StartDate,DOB,LocationCode,Current Employee Rating,Age
count,3000.000000,3000,3000,3000.000000,3000.000000,3000.000000
mean,2500.500000,2021-01-31 16:03:21.600000256,1971-09-10 11:10:04.800000,44997.180667,2.969000,53.082000
min,1001.000000,2018-08-07 00:00:00,1941-08-14 00:00:00,1013.000000,1.000000,23.000000
25%,1750.750000,2019-11-03 18:00:00,1956-06-11 18:00:00,17546.000000,2.000000,37.000000
50%,2500.500000,2021-02-08 00:00:00,1972-01-03 00:00:00,44150.500000,3.000000,53.000000
75%,3250.250000,2022-05-01 00:00:00,1987-09-09 06:00:00,71481.250000,3.000000,68.000000
max,4000.000000,2023-08-06 00:00:00,2001-07-09 00:00:00,98052.000000,5.000000,83.000000
std,866.169729,NaN,NaN,29987.331783,1.015078,17.710312


In [63]:
cleaned_file_path = "../data/employee_cleaned_data.csv"
df.to_csv(cleaned_file_path, index=False)